In [2]:
import os
import zipfile
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2

In [3]:
def read_data():
    zip_path = 'condition+monitoring+of+hydraulic+systems.zip'
    extract_path = './data'

    with zipfile.ZipFile(zip_path,'r') as zip_ref:
        zip_ref.extractall(extract_path)

    dataframes = {}
    for file in os.listdir(extract_path):
        file_path =  os.path.join(extract_path,file)

        try:
            df = pd.read_csv(file_path, delimiter='\t',header=None)
            dataframes[file] = df
            print(f'✅ successfully loaded {file}')

        except UnicodeDecodeError:
            print(f'{file} is not in csv format❌')  
    return dataframes

dataframes = read_data()

✅ successfully loaded SE.txt
✅ successfully loaded PS1.txt
✅ successfully loaded TS4.txt
✅ successfully loaded PS2.txt
✅ successfully loaded PS3.txt
✅ successfully loaded TS3.txt
✅ successfully loaded VS1.txt
✅ successfully loaded TS2.txt
✅ successfully loaded PS6.txt
✅ successfully loaded PS4.txt
documentation.txt is not in csv format❌
✅ successfully loaded TS1.txt
✅ successfully loaded PS5.txt
✅ successfully loaded CP.txt
✅ successfully loaded CE.txt
description.txt is not in csv format❌
✅ successfully loaded EPS1.txt
✅ successfully loaded FS1.txt
✅ successfully loaded FS2.txt
✅ successfully loaded profile.txt


In [4]:
#Preparing data for sql database
def rename_profile_columns():
    for key, value in dataframes.items():
        if key == 'profile.txt':
            value.columns = ['Cooler_Condition','Valve_Condition','Pump_Leakage','Accumulator_Condition','Stable_Flag']

rename_profile_columns()

def reindex_rows():
    for key, value in dataframes.items():
        index = range(1,(len(value+1)))
        dataframes[key] = value.reindex()

reindex_rows()

def check_for_missing_values():
    for key,value in dataframes.items():
        number_missing = value.isnull().sum().sum()
        print(f'{key} has {number_missing} missing values')

check_for_missing_values()

def convert_data_types():
    """Converts all columns to float if possible."""
    print('Convert data to floats')
    print('processing.....')
    for key,value in dataframes.items():
        value = value.apply(pd.to_numeric, errors="coerce")  # Convert to float, set errors as NaN
        dataframes[key] = value    
        print(f'{key} has been converted')
convert_data_types ()       



SE.txt has 0 missing values
PS1.txt has 0 missing values
TS4.txt has 0 missing values
PS2.txt has 0 missing values
PS3.txt has 0 missing values
TS3.txt has 0 missing values
VS1.txt has 0 missing values
TS2.txt has 0 missing values
PS6.txt has 0 missing values
PS4.txt has 0 missing values
TS1.txt has 0 missing values
PS5.txt has 0 missing values
CP.txt has 0 missing values
CE.txt has 0 missing values
EPS1.txt has 0 missing values
FS1.txt has 0 missing values
FS2.txt has 0 missing values
profile.txt has 0 missing values
Convert data to floats
processing.....
SE.txt has been converted
PS1.txt has been converted
TS4.txt has been converted
PS2.txt has been converted
PS3.txt has been converted
TS3.txt has been converted
VS1.txt has been converted
TS2.txt has been converted
PS6.txt has been converted
PS4.txt has been converted
TS1.txt has been converted
PS5.txt has been converted
CP.txt has been converted
CE.txt has been converted
EPS1.txt has been converted
FS1.txt has been converted
FS2.txt

In [5]:
def resample_to_100Hz():
    """
    Resamples multiple datasets to 100 Hz (6000 columns), except 'profile.txt'.

    Parameters:
    dataframes (dict): Dictionary where:
        - Keys = dataset names (e.g., "cooler", "valve", "pump")
        - Values = DataFrames (Rows = Cycle numbers, Columns = Time intervals)

    Returns:
    dict: Dictionary with resampled DataFrames.
    """
    resampled_data = {}  # Dictionary to store resampled datasets
    target_columns = 6000  # Standardized column size for 100Hz

    for key, df in dataframes.items():
        if key == 'profile.txt':  # Skip this dataset
            print(f"Skipping {key}")
            resampled_data[key] = df
            continue

        num_original_columns = df.shape[1]  # Get current number of time steps

        if num_original_columns == target_columns:
            print(f"{key} is already at 100 Hz, no resampling needed.")
            resampled_data[key] = df  # Keep original data
            continue

        # Create time indices for interpolation
        old_index = np.linspace(0, 1, num_original_columns)  # Original time scale
        new_index = np.linspace(0, 1, target_columns)  # Target time scale (6000 points)

        # Apply cubic interpolation for each row (cycle)
        resampled_df = pd.DataFrame(
            np.array([np.interp(new_index, old_index, row) for row in df.values]),
            index=df.index  # Keep original cycle numbers
        )

        resampled_data[key] = resampled_df  # Store resampled data
        print(f"{key} resampled from {num_original_columns} to {target_columns} columns.")
        print(f"New shape: {resampled_df.shape}")

    return resampled_data  # Return resampled dictionary
dataframes = resample_to_100Hz()

SE.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
PS1.txt is already at 100 Hz, no resampling needed.
TS4.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
PS2.txt is already at 100 Hz, no resampling needed.
PS3.txt is already at 100 Hz, no resampling needed.
TS3.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
VS1.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
TS2.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
PS6.txt is already at 100 Hz, no resampling needed.
PS4.txt is already at 100 Hz, no resampling needed.
TS1.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
PS5.txt is already at 100 Hz, no resampling needed.
CP.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
CE.txt resampled from 60 to 6000 columns.
New shape: (2205, 6000)
EPS1.txt is already at 100 Hz, no resampling needed.
FS1.txt resampled from 600 to 6000 columns.
New shape: (2205, 6000)
FS2.txt resampled from 600 to 6000

In [6]:
def load_data_into_DB():
    # PostgreSQL Connection
    conn = psycopg2.connect(
        dbname="predictive_maintenance",
        user="postgres",
        password="20151102",
        host="localhost",
        port="5433"
    )
    cur = conn.cursor()

    table_mapping = {
        "TS1.txt": "Temperature",
        "TS2.txt": "Temperature",
        "TS3.txt": "Temperature",
        "TS4.txt": "Temperature",
        "PS1.txt": "Pressure",
        "PS2.txt": "Pressure",
        "PS3.txt": "Pressure",
        "PS4.txt": "Pressure",
        "PS5.txt": "Pressure",
        "PS6.txt": "Pressure",
        "FS1.txt": "Flow_Sensor",
        "FS2.txt": "Flow_Sensor",
        "CE.txt": "Cooling_Efficiency",
        "SE.txt": "Stable_Efficiency",
        "CP.txt": "Cooling_Power",
        "EPS1.txt": "Efficiency_Power_Signal",
        "VS1.txt": "Vibration_Sensor",
        "profile.txt": "Profile"
    }

    # Sensor Mapping (Assign SensorIDs)
    sensor_mapping = {
        "TS1.txt": 1, "TS2.txt": 2, "TS3.txt": 3, "TS4.txt": 4,
        "PS1.txt": 5, "PS2.txt": 6, "PS3.txt": 7, "PS4.txt": 8, "PS5.txt": 9, "PS6.txt": 10,
        "FS1.txt": 11, "FS2.txt": 12
    }


    # Step 2: Populate Cycle Table
    max_cycles = max(len(df) for df in dataframes.values())  # Find max number of cycles
    cur.executemany("INSERT INTO Cycle (CycleID) VALUES (%s) ON CONFLICT DO NOTHING;", [(i,) for i in range(1, max_cycles + 1)])
    conn.commit()
    print(f"Inserted {max_cycles} rows into Cycle table")

    # Step 1: Populate the Sensor Table First**
    sensor_values = [(s_id, s_type) for s_type, s_id in sensor_mapping.items()]
    cur.executemany(
        "INSERT INTO Sensor (SensorID, Sensor_type) VALUES (%s, %s) ON CONFLICT DO NOTHING",
        sensor_values
    )
    conn.commit()
    print(f"Inserted {len(sensor_values)} rows into Sensor table.")

    # Step 3: Populate the Profile Table


    # Step 2: Insert Data into Tables
    for key, df in dataframes.items():
        if key != 'profile.txt':
            table_name = table_mapping[key]  # Convert filename to table name
            

            # Sensor-based tables (Temperature, Pressure, Flow_Sensor)
            if key in sensor_mapping:
                sensor_id = sensor_mapping[key]
                values = [
                    (idx, sensor_id, json.dumps({str(int(k.lstrip("_")) + 1): v for k, v in row._asdict().items()}))  # Convert row to JSON
                    for idx, row in enumerate(df.itertuples(index=False), start=1)  # Use row index as CycleID
                ]
                
                insert_query = f"""
                INSERT INTO {table_name} (CycleID, SensorID, {table_name}_data) VALUES (%s, %s, %s)
                """
                cur.executemany(insert_query, values)
                conn.commit()
                print(f"Inserted {len(values)} rows into {table_name}")

            # Non-Sensor tables
            else:
                values = [
                    (idx, json.dumps({str(int(k.lstrip("_")) + 1): v for k, v in row._asdict().items()}))
                    for idx, row in enumerate(df.itertuples(index=False), start=1)
                ]
                
                insert_query = f"""
                INSERT INTO {table_name} (CycleID, {table_name}_data) VALUES (%s, %s)
                """
                cur.executemany(insert_query, values)
                conn.commit()
                print(f"Inserted {len(values)} rows into {table_name}")
        else:
            values = [
            (
            idx,  # CycleID (assuming df_profile rows align with CycleID)
            int(row.Cooler_Condition),
            int(row.Valve_Condition),
            int(row.Pump_Leakage),
            int(row.Accumulator_Condition),
            bool(row.Stable_Flag)  # Convert to boolean
            )
            for idx, row in enumerate(df.itertuples(index=False), start=1)
            ]

            # SQL Insert Query
            insert_query = """
            INSERT INTO Profile (CycleID, Cooler_Condition, Valve_Condition, Pump_Leakage, Accumulator_Condition, Stable_Flag)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (CycleID) DO NOTHING;
            """

            # Execute insertion
            cur.executemany(insert_query, values)
            conn.commit()

    print(f"Inserted {len(values)} rows into Profile table.")        

    # Close Connection
    cur.close()
    conn.close()

load_data_into_DB()    

Inserted 2205 rows into Cycle table
Inserted 12 rows into Sensor table.
Inserted 2205 rows into Stable_Efficiency
Inserted 2205 rows into Pressure
Inserted 2205 rows into Temperature
Inserted 2205 rows into Pressure
Inserted 2205 rows into Pressure
Inserted 2205 rows into Temperature
Inserted 2205 rows into Vibration_Sensor
Inserted 2205 rows into Temperature
Inserted 2205 rows into Pressure
Inserted 2205 rows into Pressure
Inserted 2205 rows into Temperature
Inserted 2205 rows into Pressure
Inserted 2205 rows into Cooling_Power
Inserted 2205 rows into Cooling_Efficiency
Inserted 2205 rows into Efficiency_Power_Signal
Inserted 2205 rows into Flow_Sensor
Inserted 2205 rows into Flow_Sensor
Inserted 2205 rows into Profile table.


In [7]:
def load_into_pandas():

    # Database connection details
    conn = psycopg2.connect(
        dbname="predictive_maintenance",
        user="postgres",
        password="20151102",
        host="localhost",
        port="5433"
    )

    # Connect to PostgreSQL
    cursor = conn.cursor()

    # Define chunk size
    chunk_size = 63
    total_rows = 2205
    num_chunks = total_rows // chunk_size  # Should be 35 (2205 / 63)

    chunks = []

    for i in range(num_chunks):
        offset = i * chunk_size  # Calculate the offset for pagination
        query = f"""
            WITH 
                -- Aggregate Pressure readings by CycleID
                Pressure_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Pressure_data) AS Pressure_Readings 
                    FROM Pressure 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                ),

                -- Aggregate Temperature readings by CycleID
                Temperature_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Temperature_data) AS Temperature_Readings 
                    FROM Temperature 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                ),

                -- Aggregate Flow Sensor readings by CycleID
                Flow_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Flow_Sensor_data) AS Flow_Sensor_Readings 
                    FROM Flow_Sensor 
                    GROUP BY CycleID
                    ORDER BY CycleID
                ),

                -- Aggregate Stable Efficiency readings by CycleID
                Stable_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Stable_Efficiency_data) AS Stable_Efficiency_Readings 
                    FROM Stable_Efficiency 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                ),

                -- Aggregate Cooling Efficiency readings by CycleID
                Cooling_Eff_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Cooling_Efficiency_data) AS Cooling_Efficiency_Readings 
                    FROM Cooling_Efficiency 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                ),

                -- Aggregate Cooling Power readings by CycleID
                Cooling_Power_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Cooling_Power_data) AS Cooling_Power_Readings 
                    FROM Cooling_Power 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                ),

                -- Aggregate Vibration Sensor readings by CycleID
                Vibration_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Vibration_Sensor_data) AS Vibration_Sensor_Readings 
                    FROM Vibration_Sensor 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                ),

                -- Aggregate Efficiency Power Signal readings by CycleID
                Efficiency_Agg AS (
                    SELECT CycleID, ARRAY_AGG(Efficiency_Power_Signal_data) AS Efficiency_Power_Signal_Readings 
                    FROM Efficiency_Power_Signal 
                    GROUP BY CycleID
                    ORDER BY CycleID
                    LIMIT {chunk_size} OFFSET {offset}
                )

                SELECT 
                    c.CycleID,
                    p.Pressure_Readings,
                    t.Temperature_Readings,
                    fs.Flow_Sensor_Readings,
                    se.Stable_Efficiency_Readings,
                    ce.Cooling_Efficiency_Readings,
                    cp.Cooling_Power_Readings,
                    vs.Vibration_Sensor_Readings,
                    eps.Efficiency_Power_Signal_Readings,
                    pf.Cooler_Condition,
                    pf.Valve_Condition,
                    pf.Pump_Leakage,
                    pf.Accumulator_Condition,
                    pf.Stable_Flag
                FROM Cycle c
                INNER JOIN Pressure_Agg p ON c.CycleID = p.CycleID
                INNER JOIN Temperature_Agg t ON c.CycleID = t.CycleID
                INNER JOIN Flow_Agg fs ON c.CycleID = fs.CycleID
                INNER JOIN Stable_Agg se ON c.CycleID = se.CycleID
                INNER JOIN Cooling_Eff_Agg ce ON c.CycleID = ce.CycleID
                INNER JOIN Cooling_Power_Agg cp ON c.CycleID = cp.CycleID
                INNER JOIN Vibration_Agg vs ON c.CycleID = vs.CycleID
                INNER JOIN Efficiency_Agg eps ON c.CycleID = eps.CycleID
                INNER JOIN Profile pf ON c.CycleID = pf.CycleID;
        """

        # Execute query
        cursor.execute(query)
        
        # Fetch data
        colnames = [desc[0] for desc in cursor.description]  # Get column names
        rows = cursor.fetchall()  # Get results
        
        # Convert chunk to Pandas DataFrame
        chunk_df = pd.DataFrame(rows, columns=colnames)
        chunks.append(chunk_df)
        print(f'dataframe {i+1} has been loaded successfully')

    # Merge all chunks into a single DataFrame
    df = pd.concat(chunks, ignore_index=True)

    # Verify row count
    assert df.shape[0] == total_rows, f"Expected {total_rows} rows, got {df.shape[0]}"

    # Close connection
    cursor.close()
    conn.close()

    return df 


In [8]:
df = load_into_pandas()

dataframe 1 has been loaded successfully
dataframe 2 has been loaded successfully
dataframe 3 has been loaded successfully
dataframe 4 has been loaded successfully
dataframe 5 has been loaded successfully
dataframe 6 has been loaded successfully
dataframe 7 has been loaded successfully
dataframe 8 has been loaded successfully
dataframe 9 has been loaded successfully
dataframe 10 has been loaded successfully
dataframe 11 has been loaded successfully
dataframe 12 has been loaded successfully
dataframe 13 has been loaded successfully
dataframe 14 has been loaded successfully
dataframe 15 has been loaded successfully
dataframe 16 has been loaded successfully
dataframe 17 has been loaded successfully
dataframe 18 has been loaded successfully
dataframe 19 has been loaded successfully
dataframe 20 has been loaded successfully
dataframe 21 has been loaded successfully
dataframe 22 has been loaded successfully
dataframe 23 has been loaded successfully
dataframe 24 has been loaded successfully
d

In [25]:
df_stable = df[df['stable_flag']==True]
df_stable_reindexed = df_stable.set_index('cycleid')
df_stable_reindexed.loc[2156]

pressure_readings                   [{'1': 151.24, '2': 151.24, '3': 151.19, '4': ...
temperature_readings                [{'1': 30.621, '2': 30.62096066011002, '3': 30...
flow_sensor_readings                [{'1': 7.918, '2': 7.249504417402901, '3': 6.5...
stable_efficiency_readings          [{'1': 65.94, '2': 65.29148191365228, '3': 64....
cooling_efficiency_readings         [{'1': 46.376, '2': 46.376, '3': 46.376, '4': ...
cooling_power_readings              [{'1': 2.135, '2': 2.1350688448074675, '3': 2....
vibration_sensor_readings           [{'1': 0.552, '2': 0.5521475245874313, '3': 0....
efficiency_power_signal_readings    [{'1': 2417.6, '2': 2417.8, '3': 2417.8, '4': ...
cooler_condition                                                                  100
valve_condition                                                                   100
pump_leakage                                                                        0
accumulator_condition                                 

In [14]:
df_stable_reindexed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 756 entries, 1 to 2156
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   pressure_readings                 756 non-null    object
 1   temperature_readings              756 non-null    object
 2   flow_sensor_readings              756 non-null    object
 3   stable_efficiency_readings        756 non-null    object
 4   cooling_efficiency_readings       756 non-null    object
 5   cooling_power_readings            756 non-null    object
 6   vibration_sensor_readings         756 non-null    object
 7   efficiency_power_signal_readings  756 non-null    object
 8   cooler_condition                  756 non-null    int64 
 9   valve_condition                   756 non-null    int64 
 10  pump_leakage                      756 non-null    int64 
 11  accumulator_condition             756 non-null    int64 
 12  stable_flag               

In [26]:
df_stable_reindexed.isnull().sum()

pressure_readings                   0
temperature_readings                0
flow_sensor_readings                0
stable_efficiency_readings          0
cooling_efficiency_readings         0
cooling_power_readings              0
vibration_sensor_readings           0
efficiency_power_signal_readings    0
cooler_condition                    0
valve_condition                     0
pump_leakage                        0
accumulator_condition               0
stable_flag                         0
dtype: int64

In [30]:
df_stable_reindexed.head(10)

,pressure_readings,temperature_readings,flow_sensor_readings,stable_efficiency_readings,cooling_efficiency_readings,cooling_power_readings,vibration_sensor_readings,efficiency_power_signal_readings,cooler_condition,valve_condition,pump_leakage,accumulator_condition,stable_flag
cycleid,,,,,,,,,,,,,
1,"[{'1': 151.47, '2': 151.45, '3': 151.52, '4': ...","[{'1': 30.363, '2': 30.363118019669944, '3': 3...","[{'1': 8.99, '2': 8.169233205534256, '3': 7.34...","[{'1': 68.039, '2': 67.3698383063844, '3': 66....","[{'1': 47.202, '2': 47.20269828304717, '3': 47...","[{'1': 2.184, '2': 2.184, '3': 2.184, '4': 2.1...","[{'1': 0.604, '2': 0.6040098349724954, '3': 0....","[{'1': 2411.6, '2': 2411.6, '3': 2411.6, '4': ...",3,100,0,130,True
2,"[{'1': 151.11, '2': 151.12, '3': 151.16, '4': ...","[{'1': 33.648, '2': 33.64873762293716, '3': 33...","[{'1': 8.919, '2': 8.109815802633772, '3': 7.3...","[{'1': 68.264, '2': 67.59262543757292, '3': 66...","[{'1': 29.208, '2': 29.20420370061677, '3': 29...","[{'1': 1.414, '2': 1.4137049508251374, '3': 1....","[{'1': 0.59, '2': 0.5901966994499083, '3': 0.5...","[{'1': 2409.6, '2': 2409.6, '3': 2409.6, '4': ...",3,100,0,130,True
3,"[{'1': 150.81, '2': 150.79, '3': 150.84, '4': ...","[{'1': 35.113, '2': 35.11307867977996, '3': 35...","[{'1': 9.179, '2': 8.330674612435406, '3': 7.4...","[{'1': 68.595, '2': 67.92037006167695, '3': 67...","[{'1': 23.554, '2': 23.55367544590765, '3': 23...","[{'1': 1.159, '2': 1.1589803300550092, '3': 1....","[{'1': 0.578, '2': 0.5782458743123854, '3': 0....","[{'1': 2397.8, '2': 2397.8, '3': 2397.8, '4': ...",3,100,0,130,True
4,"[{'1': 150.48, '2': 150.47, '3': 150.52, '4': ...","[{'1': 36.133, '2': 36.132724620770134, '3': 3...","[{'1': 9.034, '2': 8.204646107684615, '3': 7.3...","[{'1': 68.628, '2': 67.95304550758459, '3': 67...","[{'1': 21.54, '2': 21.538809968328053, '3': 21...","[{'1': 1.101, '2': 1.1008524754125688, '3': 1....","[{'1': 0.565, '2': 0.5652557092848808, '3': 0....","[{'1': 2383.8, '2': 2383.8, '3': 2383.8, '4': ...",3,100,0,130,True
5,"[{'1': 150.41, '2': 150.35, '3': 150.24, '4': ...","[{'1': 36.992, '2': 36.992118019669945, '3': 3...","[{'1': 8.729, '2': 7.927803800633439, '3': 7.1...","[{'1': 68.868, '2': 68.1906851141857, '3': 67....","[{'1': 20.46, '2': 20.458406734455743, '3': 20...","[{'1': 1.086, '2': 1.085950825137523, '3': 1.0...","[{'1': 0.57, '2': 0.5702950491748624, '3': 0.5...","[{'1': 2372.0, '2': 2372.0, '3': 2372.0, '4': ...",3,100,0,130,True
6,"[{'1': 150.27, '2': 150.2, '3': 150.16, '4': 1...","[{'1': 37.824, '2': 37.824157359559926, '3': 3...","[{'1': 9.044, '2': 8.213248208034672, '3': 7.3...","[{'1': 68.972, '2': 68.29366227704617, '3': 67...","[{'1': 19.651, '2': 19.65084264044007, '3': 19...","[{'1': 1.083, '2': 1.083049174862477, '3': 1.0...","[{'1': 0.568, '2': 0.5683245540923487, '3': 0....","[{'1': 2369.6, '2': 2369.6, '3': 2369.6, '4': ...",3,100,0,130,True
7,"[{'1': 149.92, '2': 149.87, '3': 149.96, '4': ...","[{'1': 38.566, '2': 38.56596066011002, '3': 38...","[{'1': 9.393, '2': 8.528998166361061, '3': 7.6...","[{'1': 68.512, '2': 67.83818636439406, '3': 67...","[{'1': 19.339, '2': 19.33974745790965, '3': 19...","[{'1': 1.11, '2': 1.1099901650275046, '3': 1.1...","[{'1': 0.582, '2': 0.5822262043673945, '3': 0....","[{'1': 2369.8, '2': 2369.0, '3': 2368.8, '4': ...",3,100,0,130,True
8,"[{'1': 149.72, '2': 149.82, '3': 149.89, '4': ...","[{'1': 39.227, '2': 39.22787531255209, '3': 39...","[{'1': 9.109, '2': 8.27615135855976, '3': 7.44...","[{'1': 68.566, '2': 67.89165527587932, '3': 67...","[{'1': 18.788, '2': 18.78433155525921, '3': 18...","[{'1': 1.102, '2': 1.1018229704950826, '3': 1....","[{'1': 0.583, '2': 0.5833048841473578, '3': 0....","[{'1': 2355.8, '2': 2355.8, '3': 2355.8, '4': ...",3,100,0,130,True
9,"[{'1': 149.82, '2': 149.87, '3': 150.01, '4': ...","[{'1': 39.793, '2': 39.793, '3': 39.793, '4': ...","[{'1': 9.244, '2': 8.396373562260377, '3': 7.5...","[{'1': 68.697, '2': 68.02136689448241, '3': 67...","[{'1': 18.

In [28]:
print(len(df_stable_reindexed))

756


In [66]:
def split_columns():
    for column_name in df_stable_reindexed:
        if df_stable_reindexed[column_name].apply(lambda x: isinstance(x, list)).all():
            # Get the number of dictionaries in the first row (assuming all rows have the same structure)
            num_dicts = df_stable_reindexed[column_name].apply(len).unique()[0]
            df_stable_reindexed[[f'{column_name}_{i}' for i in range(1,num_dicts+1)]] = pd.DataFrame(df_stable_reindexed[column_name].tolist(),index = df_stable_reindexed.index)
            df_stable_reindexed.drop(columns=[column_name],inplace=True)        
split_columns()

(756, 22)